<a href="https://colab.research.google.com/github/futureCodersSE/python-cyber/blob/main/Cloud/Modifying_the_serverless_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The following code was used for the original serverless function
---

You are going to add functionality to this and below are some tips to help with this.

## Good practice in updating lambda functions
---

1.  **Test locally first**.  If it crashes locally then it will generate 'Internal server error' responses on Lambda and you want to avoid this.  Again, the lambda_handler function should check for as many data errors as possible to minimise the risk of crashing.  A crash could leave a database or other service exposed.  The app should always fail gracefully.  Some of this can be done with try: except: clauses which will be covered later.  

2.  The lambda_handler should sit in a file on its own and should call functions from other files.  This reduces visibility of as many functions as possible in case of crashes or unauthorised access to running code.

3.  All other functions should be imported into the lambda_handler ONLY if that function is called within the handler.

### The code should look similar to this

**Note**:  these can't be run together in a Colab notebook because they are specifically written to work in different files.

**functions.py** ( a file containing the functions that are being requested)

```
## functions.py

# a set of mathematical functions
def add_numbers(num1, num2):
  sum = num1 + num2
  return sum

def multiply_numbers(num1, num2):
  product = num1 + num2
  return product
```

**lambda_function.py** (a file containing only the *lambda_handler* function)

```
# lambda_function.py

# handle the request, passing the data to other functions

import json
from functions import add_numbers

def lambda_handler(event, context):
  # event will be in the format {"body":{"data":{{"num1":3, "num2":5}}} where 3 and 5 could be any numbers.
  # FOR SECURITY ALWAYS first check that the "body" key is there to prevent crashes.  If body is a JSON string (rather than object) convert to an object
  if "body" in event.keys():
    request = event["body"]
    if type(request) is not dict:
      request = json.loads(request)
    # check that there was some data in the body, get the values and run the function to get the result
    if request is not None and "data" in request.keys():
      # get the data from the data object
      data = request["data"]
      ### CHANGE HERE ADD A NEW KEY (operation) that will be "add" or "multiply" ###

      num1 = data["num1"]
      num2 = data["num2"]

      ### TO HERE - GETTING THE DATA TO KNOW WHAT OPERATION AND WHAT NUMBERS ###
      # now the data has been collected, it can run the right function (operation) setting return_data to the result, and set the statuscode to 200 (success)

      ### CHANGE HERE TO SELECT add_numbers if operation was "add" OR multiply_numbers if "multiply ###
      return_data = add_numbers(num1, num2)

      ### TO HERE - PERFORMING THE REQUESTED OPERATION TO SET return_data CORRECTLY ###
      
      statuscode = 200
    else:
      return_data = "Unable to get data"
      statuscode = 404
    # now it can return the result in a JSON object with some security settings in the headers
    return {
        'statusCode': statuscode,
        'headers':{
            'Content-Type' : 'application/json',
            'Access-Control-Allow-Headers' : 'Content-Type,X-Api-Key',
            'Access-Control-Allow-Methods' : 'POST',
            'Access-Control-Allow-Origin':'*'
        },
        'body': json.dumps(return_data)
    }
  ```

  
 **Initialisation file**:  `__init__.py`  it is a good idea to create a third file in the same folder called `__init__.py`.  This helps with the tracking of files as your folders get bigger and you have sub-folders.  THE FILE SHOULD BE EMPTY (although at advanced stages you may learn to add to it)

  

In [ ]:
# NO CODE IN HERE

### Get code ready to re-upload to Lambda
---

SO a folder (called **lambda_files** here) containing all the code for this lambda function should look like this:

  ```
  lambda_files  
  --> __init__.py  
  --> functions.py  
  --> lambda_function.py  
  ```
  

## BEFORE UPLOADING THE NEW CODE - TEST, TEST, TEST
---

Try the following tests:

```
request = {
    "body": {
        "data": {
            "operation":"multiply",
            "num1":5,
            "num2":3
        }
    }
}
```
```
request = {
    "body": {
        "data": {
            "num1":5,
            "num2":3
        }
    }
}
```
```
request = {
    "body": {
        "data": {
            "operation":"multiply",
            "num1":5
        }
    }
}
```

```
request = {
    "body": {}
}
```
```
request = {
    "body": {
        "data": {}
    }
}
```


In [ ]:
request = {
    "body": {
        "data": {
            "operation":"multiply",
            "num1":5,
            "num2":3
        }
    }
}

## Update the code in AWS Lambda.
---

Open your Lambda function in AWS so that you can see the existing code.  Then upload the new code (zipped as before) and test it in Lambda.

If it works, then it would be a good idea to create a collection of tests in Postman and run the whole collection to test, with an aim to find as many runtime errors as possible.

**PLEASE BE AWARE that AWS has a wide range of services, not all of them are free from cost.  If you experiment outside the services we are asking you to use, you may be charged.**